In [4]:
import numpy as np
import pandas as pd
from model.VotingModel import VotingModel

def simulate_bribery(model, target_project, desired_increase, method):
    num_voters, num_projects = model.voting_matrix.shape
    new_voting_matrix = model.voting_matrix.copy()

    # Calculate the original allocation
    original_allocation = model.allocate_funds(method)
    original_funds = original_allocation[target_project]
    
    # Calculate the target allocation
    target_funds = original_funds + desired_increase

    bribery_cost = 0
    current_increase = 0

    # Iterate to find the minimal cost to achieve the desired increase
    while current_increase < desired_increase:
        # Find the voter that can contribute the most to the target project
        for voter in range(num_voters):
            if current_increase >= desired_increase:
                break
            available_funds = new_voting_matrix[voter].sum() - new_voting_matrix[voter, target_project]
            if available_funds > 0:
                shift_amount = min(available_funds, desired_increase - current_increase)
                new_voting_matrix[voter, target_project] += shift_amount
                bribery_cost += shift_amount
                current_increase += shift_amount

    # Calculate the new allocation after bribery
    new_allocation = model.allocate_funds(method)
    
    return bribery_cost, new_voting_matrix, new_allocation

def evaluate_bribery_impact(model, target_project, desired_increase):
    # Original allocations without bribery
    original_allocations = {}
    for method in ["mean", "median", "quadratic"]:
        original_allocations[method] = model.allocate_funds(method)[target_project]
    
    # Bribery impact for each aggregation method
    bribery_results = []
    for method in ["mean", "median", "quadratic"]:
        bribery_cost, new_voting_matrix, new_allocation = simulate_bribery(
            model, target_project, desired_increase, method
        )
        bribery_results.append({
            "method": method,
            "bribery_cost": bribery_cost,
            "new_allocation": new_allocation[target_project],
            "increase": new_allocation[target_project] - original_allocations[method]
        })
    
    return pd.DataFrame(bribery_results)

# Initialize the model
model = VotingModel(num_voters=144, num_projects=600, total_op_tokens=30e6)
model.step()

# Set parameters for bribery evaluation
target_project = 0
desired_increase = 5e6  # Desired increase of 5M OP tokens

# Evaluate the impact of bribery
bribery_impact_df = evaluate_bribery_impact(model, target_project, desired_increase)
bribery_impact_df.head(10)


/Users/idrees/Code/govxs/agents/VoterAgent.py:8: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  super().__init__(unique_id, model)


,method,bribery_cost,new_allocation,increase
0,mean,5000000.0,48292.335447,0.0
1,median,5000000.0,45746.102493,0.0
2,quadratic,5000000.0,46329.662680,0.0
